In [1]:
import numpy as np
from mprod import  m_prod
from mprod import  generate_haar, generate_dct, x_m3
from scipy.stats import ortho_group
from scipy.fft import dct, idct, rfft, irfft

In [2]:
m, n = 100, 10
shape = (m, n)
np.random.seed(42)

R = np.random.uniform(-1, 1, shape).reshape(m, 1, n)
I = np.random.uniform(-1, 1, shape).reshape(m, 1, n)
A = R+ 1.j * I

In [3]:
A.shape

(100, 1, 10)

In [5]:
def tensor_hermit(tensor):
    m, p, n = tensor.shape
    Rtensor = tensor.real
    Itensor = tensor.imag
    hermitian_tensor = np.empty((p,m,n), dtype=np.csingle)
    for d in range(n):
        hermitian_tensor[:,:,d] = Rtensor[:,:,d].T - 1.j * Itensor[:,:,d].T
    
    return hermitian_tensor

In [6]:
A_H = tensor_hermit(A)
print(A.shape)
print(A_H.shape)

(100, 1, 10)
(1, 100, 10)


In [ ]:
A_H[:,0]

array([[-0.25091976+0.62973416j,  0.90142864-0.0838019j ,
         0.4639879 -0.7458917j ,  0.19731697-0.46444976j,
        -0.6879627 -0.6131223j , -0.68801093-0.31756672j,
        -0.88383275-0.38455313j,  0.7323523 -0.6983913j ,
         0.20223002+0.500664j  ,  0.41614515+0.02115007j]],
      dtype=complex64)

In [9]:
funM, invM = generate_haar(n, random_state=21)

In [ ]:
funM(m_prod(A_H, A, funM, invM)).real

array([[[66.05192715, 65.56827073, 57.91509212, 72.73006634,
         59.04925305, 70.54821916, 68.99981625, 75.5851514 ,
         71.80348141, 74.51924321]]])

In [ ]:
funM(m_prod(A_H, A, funM, invM)).imag

array([[[ 9.58427889e-08, -1.10931771e-08,  1.96089945e-07,
          8.02648579e-08, -9.59434035e-08,  4.41943619e-08,
         -1.64936129e-07, -1.00774381e-07,  2.19106113e-07,
          1.79103076e-08]]])

## to lessen numercal error I am constracting m_prod_H function where I use algebraic formula $(a+b)(a-b)=a^2-b^2$, then $(a+ib)(a-ib)=a^2+b^2$

In [12]:
def m_prod_H(tensor, funM, invM):
    tensor_hat = funM(tensor)
    tensor_hat_H = tensor_hermit(tensor_hat)
    real_square = np.einsum('mpi,pli->mli', tensor_hat_H.real, tensor_hat.real)
    imag_square = -np.einsum('mpi,pli->mli', tensor_hat_H.imag, tensor_hat.imag)
    prod_M_hat = real_square+imag_square
    return invM(prod_M_hat)

In [13]:
funM(m_prod_H(A, funM, invM))>0

array([[[ True,  True,  True,  True,  True,  True,  True,  True,  True,
          True]]])

In [14]:
funM(m_prod_H(A, funM, invM)).shape

(1, 1, 10)

In [15]:
def T_norm(matrix_v, funM, invM):
    tuple_sq = funM(m_prod_H(matrix_v, funM, invM)).squeeze()
    frob_norm = np.sqrt(np.sum(tuple_sq))
    return frob_norm

In [16]:
def frob_norm_complex(tensor):
    return np.sqrt(np.sum(tensor.real**2+tensor.imag**2))

In [17]:
T_norm(A, funM, invM)

26.129877932029803

In [18]:
frob_norm_complex(A)

26.129877933562025

# numericaly these norms are equal